# Pulling + Organizing Data

In [1]:
# Import packages
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import sqlite3

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)

## Election Results

In [2]:
# Import data
data1 = pd.read_excel("statistic_id1034688_share-of-electoral-and-popular-votes-by-each-united-states-president-1789-2020 (1).xlsx", sheet_name = "Data")
# Clean the data
results = data1[4:]
results.rename(columns={'Unnamed: 0': 'x1', 'Unnamed: 1':'President', 'Unnamed: 2': 'Electoral College', 'Unnamed: 3': 'Popular Vote', 'Unnamed: 4': 'x2'}, inplace=True)
results.drop(['x1'], axis = 1, inplace = True)
results.drop(['x2'], axis = 1, inplace = True)
results = results.replace('-', np.nan)
# Get the year
results[['President', 'Year']] = results.President.str.split(pat = '(', n = 1, expand = True)
results.Year = results.Year.str[:-1]
# Reverse order
results = results[::-1]
results.reset_index(drop = True, inplace = True)
# Set column types
results = results.astype({'Year': int, 'Electoral College': float, 'Popular Vote': float})
results = results[['Year', 'President', 'Electoral College', 'Popular Vote']]

# Remove space from end of President name
results['President'] = results['President'].str[:-1]

results

,Year,President,Electoral College,Popular Vote
0,1789,George Washington,100.00,NaN
1,1792,George Washington,100.00,NaN
2,1796,John Adams,51.45,NaN
3,1800,Thomas Jefferson,52.90,NaN
4,1804,Thomas Jefferson,92.05,NaN
5,1808,James Madison,69.71,NaN
6,1812,James Madison,58.99,NaN
7,1816,James Monroe,84.33,NaN
8,1820,James Monroe,98.30,NaN
9,1824,John Quincy Adams,32.18,30.9


## Previous Presidents

In [16]:
def standardize_party(string):
    if string == "Unaffiliated":
        string = "GW"
    if string == "Federalist":
        string = "F"
    if string == "Democratic-Republican":
        string = "DR"
    if string == "Democratic":
        string = "D"
    if string == "Whig":
        string = "W"
    if string == "Republican":
        string = "R"
    return string

In [17]:
# Pull data
url = "http://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States"
content = pd.read_html(url)
prev_pres = pd.concat(content)
prev_pres = prev_pres[10:92]

# Drop columns
prev_pres.drop(columns =[0, 1, 'Party[b]', 'Politics of the United States', 
                 'Portrait', 'Vice President','vteChief executives of the United States', 
                 'vteChief executives of the United States.1', 
                 'vteLists related to the presidents and vice presidents of the United States', 
                 'vteLists related to the presidents and vice presidents of the United States.1', 
                 'vtePresidents of the United States', 'vtePresidents of the United States.1', 
                 'Presidency[a]', 'Presidency[a].1'],axis = 1, inplace = True)
prev_pres.drop_duplicates(inplace = True)
# Get rid of rows where a President changes party midway through
prev_pres.drop_duplicates(subset = ['Election', 'Name(Birth–Death)'], keep='first', inplace = True)
# Get rid of rows where a President wasn't elected - started in position from assassination of previous
prev_pres.drop_duplicates(subset = ['Election'], keep = 'first', inplace = True)

# Clean column content
prev_pres['Name(Birth–Death)'] = prev_pres['Name(Birth–Death)'].str.split('(').str[0]
prev_pres['Name(Birth–Death)'] = prev_pres['Name(Birth–Death)'].str.split('[').str[0]
prev_pres.rename(columns ={'Election': 'Year', 'Name(Birth–Death)': 'Name', 'Party[b].1': 'Party'}, inplace = True)
prev_pres.Year[0] = '1789'
prev_pres = prev_pres.astype({'Year': int})

# Change party to be shortened form
prev_pres.Party = prev_pres.Party.apply(standardize_party)
# Fix individual places
prev_pres['Party'][11] = "Adams"
prev_pres['Party'][27] = "R"

In [18]:
prev_pres.head()

,Year,Name,Party
0,1789,George Washington,GW
1,1792,George Washington,GW
2,1796,John Adams,F
3,1800,Thomas Jefferson,DR
4,1804,Thomas Jefferson,DR


## Economic Growth

In [19]:
# Read in data
data2 = pd.read_excel("statistic_id996758_annual-gdp-growth-for-the-united-states-1930-2020 (1).xlsx", sheet_name = "Data")

In [20]:
# Get necessary columns and fix types
gdp = data2[4:]
gdp.reset_index(drop = True, inplace = True)
gdp.drop(columns = ['Unnamed: 0', 'Unnamed: 3'], axis = 1, inplace = True)
gdp.rename(columns = {'Unnamed: 1': "Year", "Unnamed: 2": "Change in GDP in %"}, inplace = True)
gdp = gdp.astype({'Year': int, "Change in GDP in %": float})

# Only look at election years
gdp = gdp[gdp['Year'] % 4 == 0]
gdp.head()

,Year,Change in GDP in %
2,1932,-12.9
6,1936,12.9
10,1940,8.8
14,1944,8.0
18,1948,4.1


## State votes

In [21]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state'
content = pd.read_html(url)

states = pd.concat(content)
states.columns = states.iloc[0]
states.drop(states.index[0], inplace = True)

states = states[0: 52]
states = states.iloc[:, 0: 66]
states = states.loc[:, states.columns.notnull()]
states = states.set_index("State")

# Transpose 
states.drop("State", axis = 0, inplace = True)
states = states.transpose()
states.reset_index(inplace = True)
states.rename(columns = {0: "Year"}, inplace = True)
states.Year = states.Year.str[0:4]
states = states.astype({'Year': int})

states = states.merge(prev_pres[['Year', 'Party']], how="left",on= "Year")
states.head()

,Year,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,D.C.,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Party
0,1789,NaN,NaN,NaN,NaN,NaN,NaN,GW,GW,NaN,...,NaN,NaN,NaN,NaN,GW,NaN,NaN,NaN,NaN,GW
1,1792,NaN,NaN,NaN,NaN,NaN,NaN,GW,GW,NaN,...,NaN,NaN,NaN,GW,GW,NaN,NaN,NaN,NaN,GW
2,1796,NaN,NaN,NaN,NaN,NaN,NaN,F,F,NaN,...,DR,NaN,NaN,F,DR,NaN,NaN,NaN,NaN,F
3,1800,NaN,NaN,NaN,NaN,NaN,NaN,F,F,NaN,...,DR,NaN,NaN,F,DR,NaN,NaN,NaN,NaN,DR
4,1804,NaN,NaN,NaN,NaN,NaN,NaN,F,F,NaN,...,DR,NaN,NaN,DR,DR,NaN,NaN,NaN,NaN,DR


In [22]:
# Overall swing state list

# Create empty list
state_accuracy = list()
for state in range(1, 52):
    # Find number of times a state has voted for the same party as the winning party, divide by number of times a state has voted
    state_accuracy.append([(np.sum(states.iloc[:,state] == states.Party))/states.iloc[:,state].count(), states.columns[state]])
state_accuracy.sort(reverse = True)
state_accuracy[0:7]

[[0.8928571428571429, 'New Mexico'],
 [0.8235294117647058, 'Illinois'],
 [0.8181818181818182, 'Ohio'],
 [0.813953488372093, 'California'],
 [0.8135593220338984, 'Pennsylvania'],
 [0.8103448275862069, 'New York'],
 [0.8, 'Nevada']]

In [23]:
# More accurate swing state list

# Create empty list
state_accuracy_better = list()
for state in range(1, 52):
    # Find number of times a state has voted for the same party as the winning party, weighting the most recent 8 years as 10 times more significant
    num_correct = np.sum(states.iloc[:51, state] == states.iloc[:51,:].Party)
    num_correct += 10*(np.sum(states.iloc[51:, state] == states.iloc[51:,:].Party))
    # Divide by number of times a state has voted + 10*8 = 80
    state_accuracy_better.append([num_correct/(states.iloc[:,state].count() + 80), states.columns[state]])
state_accuracy_better.sort(reverse = True)
state_accuracy_better[0:7]

[[0.8, 'Ohio'],
 [0.7916666666666666, 'Nevada'],
 [0.7338129496402878, 'Pennsylvania'],
 [0.7314814814814815, 'New Mexico'],
 [0.7096774193548387, 'Wisconsin'],
 [0.7050359712230215, 'New Hampshire'],
 [0.7007874015748031, 'Michigan']]

The states that are the most predictive(historically) are New Mexico, Illinois, Ohio, California, Pennsylvania, New York, and Nevada(all 80% accuracy or above). Taking into account that some states have voted differently the past several years, we make a new, more accurate list of swing states that more accurately reflects what we typically think of as swing states: Ohio, Nevada, Pennslyvania, New Mexico, Wisconsin, New Hampshire, and Michigan. We can make all of these into dummy variables to get both a historical and current sense of important states for the full dataset.

In [24]:
swing_states_overall = ['New Mexico', 'Illinois', 'Ohio', 'California', 'Pennsylvania', 'New York', 'Nevada']
swing_states_accurate = ['Ohio', 'Nevada', 'Pennsylvania', 'New Mexico', 'Wisconsin', 'New Hampshire', 'Michigan']

In [25]:
# Create dummy variables for states
key_states = ['New Mexico', 'Illinois', 'Ohio', 'California', 'Pennsylvania', 'New York', 'Nevada', 'Wisconsin', 'New Hampshire', 'Michigan']
state_dummies = states[key_states]
state_dummies[['Year','Party']] = states[['Year','Party']]

for state in key_states:
    state_dummies[state] = 1*(state_dummies[state] == state_dummies['Party'])

state_dummies.drop(columns = 'Party', inplace = True)
state_dummies.head()

,New Mexico,Illinois,Ohio,California,Pennsylvania,New York,Nevada,Wisconsin,New Hampshire,Michigan,Year
0,0,0,0,0,1,0,0,0,1,0,1789
1,0,0,0,0,1,1,0,0,1,0,1792
2,0,0,0,0,0,1,0,0,1,0,1796
3,0,0,0,0,1,1,0,0,0,0,1800
4,0,0,1,0,1,1,0,0,1,0,1804


## Turnout

In [26]:
url = 'https://docs.google.com/spreadsheets/d/1bH38j6_e8yA9xq8OMlyLOL6h_iTS7ABQMKNxzFgKBDo'
content = pd.read_html(url)
data = pd.concat(content)

In [27]:
turnout = data.iloc[:, 5:7]
turnout.rename(columns = {'A': 'Year', 'B': 'Turnout Rate'}, inplace = True)
turnout.drop(0, axis = 0, inplace = True)
turnout.reset_index(drop = True, inplace = True)
# Add turnout rate for 2020 
turnout = turnout.append({'Year': 2020, 'Turnout Rate': 66.9}, ignore_index = True)
turnout = turnout.astype({'Year': int, 'Turnout Rate': float})
turnout.tail()

,Year,Turnout Rate
54,2004,60.1
55,2008,61.6
56,2012,58.6
57,2016,60.1
58,2020,66.9


# Combining Data

In [28]:
# Combine current things - not looking at stats about previous year's presidents
# Election Results, Economic Growth, State Dummies, Turnout, Party
results, gdp, state_dummies, turnout
total_data = pd.merge(left = results, right = gdp, how = 'outer',on = "Year")
total_data = pd.merge(left = total_data, right = turnout, how = 'outer', on = 'Year')
total_data = pd.merge(left = total_data, right = prev_pres[['Year', 'Party']], how = 'outer', on = 'Year')
total_data = pd.merge(left = total_data, right = state_dummies, how = 'outer', on = 'Year')

# For combining info on previous presidents, will have to make year = year - 4 in prev_pres and ratings
prev_pres['current_year'] = prev_pres['Year'] + 4
# Fix first year
prev_pres['current_year'][0] = 1792

# Combine
total_data = pd.merge(left = total_data, right = prev_pres[['current_year', 'Name', 'Party']], 
                      how = 'left', left_on = "Year", right_on = "current_year")

# Drop + rename
total_data.drop(columns = 'current_year', inplace = True)
total_data.rename(columns = {'Party_x': 'Party', 'Name': 'Prev_President', 'Party_y': 'Prev_Pres_Party'}, inplace = True)

In [29]:
# Add variables relating to previous president

# Are they an incumbent?
total_data['Incumbent'] = 1*(total_data.President == total_data.Prev_President)
# Were they the same party?
total_data['Same_party_as_prev'] = 1*(total_data.Party == total_data.Prev_Pres_Party)
# Same party but not incumbent
total_data['Same_party_not_incum'] = 1*((total_data.Same_party_as_prev == 1) & (total_data.Incumbent == 0))

# Drop columns
total_data.drop(columns = ['Prev_President', 'Prev_Pres_Party'], inplace = True)

In [30]:
total_data

,Year,President,Electoral College,Popular Vote,Change in GDP in %,Turnout Rate,Party,New Mexico,Illinois,Ohio,California,Pennsylvania,New York,Nevada,Wisconsin,New Hampshire,Michigan,Incumbent,Same_party_as_prev,Same_party_not_incum
0,1789,George Washington,100.00,NaN,NaN,11.6,GW,0,0,0,0,1,0,0,0,1,0,0,0,0
1,1792,George Washington,100.00,NaN,NaN,6.3,GW,0,0,0,0,1,1,0,0,1,0,1,1,0
2,1796,John Adams,51.45,NaN,NaN,20.1,F,0,0,0,0,0,1,0,0,1,0,0,0,0
3,1800,Thomas Jefferson,52.90,NaN,NaN,32.3,DR,0,0,0,0,1,1,0,0,0,0,0,0,0
4,1804,Thomas Jefferson,92.05,NaN,NaN,23.8,DR,0,0,1,0,1,1,0,0,1,0,1,1,0
5,1808,James Madison,69.71,NaN,NaN,36.8,DR,0,0,1,0,1,1,0,0,0,0,0,1,1
6,1812,James Madison,58.99,NaN,NaN,40.4,DR,0,0,1,0,1,0,0,0,0,0,1,1,0
7,1816,James Monroe,84.33,NaN,NaN,16.9,DR,0,0,1,0,1,1,0,0,1,0,0,1,1
8,1820,James Monroe,98.30,NaN,NaN,10.1,DR,0,1,1,0,1,1,0,0,1,0,1,1,0
9,1824,John Quincy Adams,32.18,30.9,NaN,26.9,Adams,0,0,0,0,0,1,0,0,1,0,0,0,0


## SQL Database connection

In [32]:
# Create SQL database to store dataframes
conn = sqlite3.connect("Final_Project.sqlite")
results.to_sql(name="Election_Results",con=conn,index=False)
prev_pres.to_sql(name="Previous_Presidents",con=conn,index=False)
gdp.to_sql(name="Econ",con=conn,index=False)
state_dummies.to_sql(name="States",con=conn,index=False)
turnout.to_sql(name="Turnout",con=conn,index=False)
total_data.to_sql(name="Total_Data",con=conn,index=False)
conn.close()